## Demographics

a. Identify IDs with 3T and 7T  
b. Extract clinical information for epilepsy patients  
c. Extract demographic information for all participants  


Note participant grouping:
- CTRL : no known epilepsy (identified by ID code: 3T: HC; 7T: PNC, Pilot)

- (m)TLE : temporal lobe epilepsy
    - TLE_L : TLE left lateralized (including L>R)
    - TLE_R : TLE right lateralized (including R>L)
    - TLE_U : TLE with unknown lateralization
    - mTLE_L : mTLE left lateralized
    - mTLE_R : mTLE right lateralized
- FLE : focal epilepsy that is not in temporal lobe
    - FLE_L
    - FLE_R
- UKN : Focal epilepsy with focus in unknown lobe
    - UKN_L : Unknown lobe with known L lateralization
    - UKN_R : Unknown lobe with known R lateralization
    - UKN_U : Unknown lobe with unknown lateralization
- MFCL : multifocal epilepsy
    - MFCL : multifocal epilepsy


In [266]:
import pandas as pd
import os
import sys
import importlib
import re
import numpy as np

sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code")
#sys.path.append("/host/verges/tank/data/daniel/")
from Utils import id, gen, t1

In [267]:
importlib.reload(id)
importlib.reload(gen)
importlib.reload(t1)

<module 'Utils.t1' from '/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code/Utils/t1.py'>

# I. Functions

In [ ]:
def dupSES(df, uniqCols, mergeCols):
    """
    Identify and resolve repeated session codes for the same participant at the same study

    Input:
        df: DataFrame containing the session data
        uniqCols: List of columns whose combination should be unique in each row
        mergeCols: List of columns whose values may need merging (eg., scanDate)

    Return:
        DataFrame with resolved repeated sessions
    """
    
    if df.duplicated(subset=uniqCols).any():
        n_repeated = df.duplicated(subset=uniqCols, keep=False).groupby([df[c] for c in uniqCols]).any().sum()
        print(f"[dupSES] WARNING: There are {n_repeated} participant-study combinations with repeated rows:")
        dup_rows = df[df.duplicated(subset=uniqCols, keep=False)]
        grouped = dup_rows.groupby(['MICS_ID', 'PNI_ID', 'study', 'SES']).size().reset_index(name='count')
        for _, row in grouped.iterrows():
            # Get all rows for this individual/session
            rows = dup_rows[
                (dup_rows['MICS_ID'] == row['MICS_ID']) &
                (dup_rows['PNI_ID'] == row['PNI_ID']) &
                (dup_rows['study'] == row['study']) &
                (dup_rows['SES'] == row['SES'])
            ]
            merge_info = []
            for col in mergeCols:
                vals = rows[col].dropna().unique()
                # Conflict resolution logic
                if len(vals) > 1:
                    # If column name contains 'date', resolve by earliest date
                    if 'date' in col.lower():
                        # Try to parse dates, fallback to string sort if fails
                        try:
                            parsed = pd.to_datetime(vals, errors='coerce', dayfirst=True) # d.m.y format
                            chosen = vals[parsed.argmin()] if parsed.notna().any() else sorted(vals)[0]
                        except Exception:
                            chosen = sorted(vals)[0]
                        method = "earliest date"
                    else:
                        # Default: use first value (could add more strategies)
                        chosen = sorted(vals)[0]
                        method = "first (sorted) value"
                    # Update all rows for this group to use the chosen value
                    df.loc[
                        (df['MICS_ID'] == row['MICS_ID']) &
                        (df['study'] == row['study']) &
                        (df['SES'] == row['SES']),
                        col
                    ] = chosen
                else:
                    chosen = vals[0] if len(vals) > 0 else None
                    method = "unique/no conflict"
                merge_info.append(f"{col}={list(vals)} -> {chosen} ({method})")
            merge_str = "; ".join(merge_info)
            print(f"\t[{row['MICS_ID']}={row['PNI_ID']}-{row['study']}] ses-{row['SES']} (x{row['count']}): {merge_str}")
        
        df = df.drop_duplicates(subset=uniqCols, keep='first').reset_index(drop=True) # remove duplicated rows

    return df

def rmvNADate(df, dateCol):
    """
    Remove rows with missing scan date from the DataFrame.

    Input:
        df: DataFrame to process
        dateCol: Name of the date column to check for missing values
    """
    n_missing_dates = df[dateCol].isna().sum()
    if n_missing_dates != 0:
        missing_dates = df[df[dateCol].isna()]
        print(missing_dates[['MICS_ID', 'study', dateCol]])
        count_7T = (missing_dates['study'] == '7T').sum()
        count_3T = (missing_dates['study'] == '3T').sum()
        print(f"\n[rmvNADate] WARNING. {n_missing_dates} rows have missing scan dates:\n\t{count_7T} from 7T\n\t{count_3T} from 3T")
        print("[rmvNADate] Removing the following participants (study of the removed row):")
        missing_ids = missing_dates['MICS_ID'].unique()
        df = df.dropna(subset=[dateCol])
        for mid in missing_ids:
            # there should be at least one row with each study code
            studies = df[df['MICS_ID'] == mid]['study'].unique() # remaining studies for that ID
            if len(studies) < len(df['study'].unique()): # if this has ID has fewer unique studies than those present in the data, remove all rows for that ID
                print(f"\t{mid} (removed study: {studies})")
                df = df[df['MICS_ID'] != mid]
    else:
        print("\n[rmvNADate] No rows with missing scan dates found.")

    return df

def mergeCols(df):
    """
    Merge columns with similar names (case-insensitive, ignoring whitespace) by prioritizing non-null values.
    For each group of similar columns, select the column with the shortest name (or lowercase if tied),
    and fill its missing values with values from the other columns in the group (row-wise).
    Drops the other columns in the group.

    Special logic: If both 'date' and 'scanDate' columns exist (case-insensitive), merge them into a single 'date' column,
    preferring non-null and earliest date values, and print out the merge process.

    Input:
        df: DataFrame with potential duplicate/similar columns

    Output:
        df: DataFrame with merged columns
    """


    # Normalize column names: lower case, strip whitespace
    def norm(col):
        return re.sub(r'\s+', '', col).lower()

    cols = list(df.columns)
    norm_map = {}
    for col in cols:
        key = norm(col)
        norm_map.setdefault(key, []).append(col)

    # Special handling for 'date' and 'scanDate'
    date_cols = [c for c in cols if norm(c) in ('date', 'scandate')]
    if len(date_cols) >= 2:
        # Only proceed if both columns exist
        main_col = [c for c in date_cols if norm(c) == 'date']
        scan_col = [c for c in date_cols if norm(c) == 'scandate']
        if main_col and scan_col:
            main_col = main_col[0]
            scan_col = scan_col[0]
            # If 'date' column does not exist, create it
            if 'date' not in df.columns:
                df['date'] = np.nan
            # Only operate on rows where values differ and both are not null
            mask = (df[main_col].notna() & df[scan_col].notna() & (df[main_col] != df[scan_col]))
            if mask.any():
                print(f"[mergeCols] Merging '{main_col}' and '{scan_col}' into 'date' for rows where values differ")
                for idx, row in df[mask].iterrows():
                    val1 = row[main_col]
                    val2 = row[scan_col]
                    d1 = pd.to_datetime(val1, errors='coerce', dayfirst=True)
                    d2 = pd.to_datetime(val2, errors='coerce', dayfirst=True)
                    chosen = None
                    method = ""
                    if pd.notna(d1) and pd.notna(d2):
                        if d1 <= d2:
                            chosen = val1
                            method = "earliest (date)"
                        else:
                            chosen = val2
                            method = "earliest (scanDate)"
                    elif pd.notna(d1):
                        chosen = val1
                        method = "date only"
                    elif pd.notna(d2):
                        chosen = val2
                        method = "scanDate only"
                    else:
                        chosen = val1 if pd.notna(val1) else val2
                        method = "non-date fallback"
                    df.at[idx, 'date'] = chosen
                    # Use .get to avoid KeyError for missing columns
                    mics_id = row.get('MICS_ID', 'NA')
                    pni_id = row.get('PNI_ID', 'NA')
                    study = row.get('study', 'NA')
                    ses = row.get('SES', 'NA')
                    print(f"\t[mergeCols] [{mics_id}={pni_id}-{study}] ses-{ses}: {main_col}={val1}, {scan_col}={val2} -> {chosen}")
            # For all other rows, fill 'date' with available value (prefer main_col, then scan_col)
            for idx, row in df.iterrows():
                if pd.isna(df.at[idx, 'date']):
                    val1 = row[main_col]
                    val2 = row[scan_col]
                    df.at[idx, 'date'] = val1 if pd.notna(val1) else val2
            # Drop both original columns except 'date'
            drop_cols = [c for c in [main_col, scan_col] if c != 'date']
            df.drop(columns=drop_cols, inplace=True)
            # Remove from norm_map so not merged again
            for c in drop_cols:
                norm_map.pop(norm(c), None)
            if main_col != 'date':
                norm_map.pop(norm(main_col), None)
            if scan_col != 'date':
                norm_map.pop(norm(scan_col), None)

    # Merge other similar columns
    for group in norm_map.values():
        if len(group) > 1:
            # Skip if group contains both 'date' and 'scanDate' (already handled)
            if set([norm(c) for c in group]) == set(['date', 'scandate']):
                continue
            # Choose the column with shortest name, or lowercase if tied
            group_sorted = sorted(group, key=lambda x: (len(x), x.lower()))
            main_col = group_sorted[0]
            other_cols = [c for c in group if c != main_col]
            print(f"[mergeCols] Merging columns: {group} -> '{main_col}' (date/scandate in group: {any(norm(c) in ['date', 'scandate'] for c in group)})")
            # Fill missing values in main_col from other columns, row-wise
            df[main_col] = df[group].bfill(axis=1).iloc[:, 0]
            # Drop the other columns
            df.drop(columns=other_cols, inplace=True)
    return df

def carryVals(df, var):
    """
    Carry forward and backward fill demographic variables for each participant.

    For each participant (grouped by 'MICS_ID'), if multiple unique non-null values are found for the variable:
      - Use the value with the shortest number of characters.
      - If there is a tie, use the first value encountered.
      - Print a warning indicating all found values and which one was used, including study and session number if available.

    Input:
        df: DataFrame containing demographic information
        var: Column name to carry forward/backward fill

    Output:
        df: DataFrame with the variable filled per participant
    """
    df[var] = df.groupby('MICS_ID')[var].transform(lambda x: x.ffill().bfill())
    # Check for multiple unique values for the same ID
    dupes = df.groupby('MICS_ID')[var].nunique()
    multi_val_ids = dupes[dupes > 1].index
    for mid in multi_val_ids:
        vals = df[df['MICS_ID'] == mid][var].dropna().unique()
        # Choose value with shortest number of characters, if tie, use first
        vals_sorted = sorted(vals, key=lambda v: (len(str(v)), str(v)))
        chosen = vals_sorted[0]
        pni_id = df[df['MICS_ID'] == mid]['PNI_ID'].iloc[0] if 'PNI_ID' in df.columns else ''
        study = df[df['MICS_ID'] == mid]['study'].iloc[0] if 'study' in df.columns else ''
        ses = df[df['MICS_ID'] == mid]['SES'].iloc[0] if 'SES' in df.columns else ''
        used_idx = list(vals).index(chosen) + 1
        print(f"\t[carryVals] WARNING: [{mid}={pni_id} Study: {study}-ses{ses}] {var} : {' | '.join(map(str, vals))} --> {chosen}")
    
    # Use the chosen value for each group
    def choose_val(x):
        non_nulls = x.dropna()
        if len(non_nulls.unique()) > 1:
            vals_sorted = sorted(non_nulls.unique(), key=lambda v: (len(str(v)), str(v)))
            return vals_sorted[0]
        elif len(non_nulls) > 0:
            return non_nulls.iloc[0]
        else:
            return x
    df[var] = df.groupby('MICS_ID')[var].transform(choose_val)
    return df

def group(df, ID_col="MICS_ID", out_col="grp", save_pth=None, MFCL_col="Epilepsy classification:Focal,Generalized", lobe_col="Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)", lat_col="Lateralization of epileptogenic focus"):
    """
    Requires pandas as pd

    Inputs:
    df: str or pd.DataFrame
        demographics data
    ID_col: str
        Column name for the ID
    out_col: str
        Column name for the output group classification. 
        Options: 'grp' returns high-level grouping. All other col_names return detailed grouping.
    MFCL_col: str
        Column name for the multifocal classification
    lobe_col: str  
        Column name for the lobe classification
    lat_col: str
        Column name for the lateralization classification

    Outputs:
    df: pd.DataFrame
        DataFrame with the group classification added
    """

    import pandas as pd
    import os

    print("[group] Identifying participant groups")
    # check if df is a string (path) or dataframe
    if isinstance(df, str):
        # check if file exists
        if not os.path.isfile(df):
            raise ValueError(f"[group] Error: {df} does not exist.")
        
        # read in file
        df = pd.read_csv(df, dtype=str)
    elif isinstance(df, pd.DataFrame):
        pass
    else:
        raise ValueError("[group] Error: df must be a string (path) or dataframe")

    df[out_col] = df.apply(
        lambda row: f"PATTERN NOT RECOGNIZED: lobe={row.get(lobe_col, None)}, lat={row.get(lat_col, None)}, MFCL={row.get(MFCL_col, None)}",
        axis=1
    )

    if ID_col == "MICS_ID":
        ctrl_ptrn = ["HC"]
    elif ID_col == "PNI_ID":
        ctrl_ptrn = ["Pilot", "PNC"]
    else:
        raise ValueError("[group] Error: ID_col must be 'MICS_ID' or 'PNI_ID'")

    
    if out_col == "grp":
        print("\tReturning highlevel grouping to column: ", out_col)
       
        df.loc[df[ID_col].astype(str).str.contains('|'.join(ctrl_ptrn), na=False), out_col] = 'CTRL'
        
        # TLE and mTLE: L, left, R, right, unclear
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & 
            (df[lat_col].astype(str).str.lower().str.contains('l|left|r|right|l>r|r>l|bl|bilateral|unclear', na=False)), 
            out_col
        ] = 'TLE'
        
        # FLE: L, R, right, unclear
        df.loc[
            (df[lobe_col] == 'FLE') & 
            (df[lat_col].astype(str).str.contains('l|r|right|unclear', case=False, na=False)), 
            out_col
        ] = 'FLE'
        
        # PLE: L, R, right, unclear
        df.loc[
            ((df[lobe_col] == 'PLE')) &
            (df[lat_col].astype(str).str.contains('l|r|right|unclear', case=False, na=False)),
            out_col
        ] = 'PLE'

        # PQLE: L, R, right, unclear
        df.loc[
            ((df[lobe_col] == 'PQLE')) &
            (df[lat_col].astype(str).str.contains('l|r|right|unclear', case=False, na=False)),
            out_col
        ] = 'PQLE'

        # Unclear: L, R, right, unclear
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col] == 'L'), out_col] = 'UKN'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')), out_col] = 'UKN'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col].astype(str).str.contains('unclear', na=False)), out_col] = 'UKN'
        
        # Multifocal
        df.loc[(df[MFCL_col] == 'Multifocal'), out_col] = 'MFCL'
    
    else:  
        print("\tReturning detailed grouping to column: ", out_col) 
        
        df.loc[df[ID_col].astype(str).str.contains('|'.join(ctrl_ptrn), na=False), out_col] = 'CTRL'
        
        # Combine TLE and mTLE, label as TLE
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & (df[lat_col] == 'L'),
            out_col
        ] = 'TLE_L'
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')),
            out_col
        ] = 'TLE_R'
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & (df[lat_col].astype(str).str.contains('unclear', na=False)),
            out_col
        ] = 'TLE_U'

        df.loc[(df[lobe_col] == 'FLE') & (df[lat_col] == 'L'), out_col] = 'FLE_L'
        df.loc[(df[lobe_col] == 'FLE') & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')), out_col] = 'FLE_R'

        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col] == 'L'), out_col] = 'UKN_L'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')), out_col] = 'UKN_R'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col].astype(str).str.contains('unclear', na=False)), out_col] = 'UKN_U'
        
        df.loc[(df[MFCL_col] == 'Multifocal'), out_col] = 'MFCL'
        df.loc[(df[lobe_col] == 'TLE') & (df[lat_col] == 'L>R'), out_col] = 'TLE_L'
        df.loc[(df[lobe_col] == 'TLE') & (df[lat_col] == 'R>L'), out_col] = 'TLE_R'
        df.loc[(df[lobe_col] == 'TLE') & (df[lat_col] == 'BL'), out_col] = 'TLE_BL'
    
    return df

def grpChk(df, grp_cols=['grp', 'grp_detailed']):
    """
    Check that all cases are assigned a group.
    If not, print warning

    Input:
        df: DataFrame to check
        grp_col: List of group columns to check
    """
    missing = []
    for col in grp_cols:
        df_missing = df[df[col] == ''][["MICS_ID", "PNI_ID", "SES", col]]
        if not df_missing.empty:
            missing.append(df_missing)  # Use append, not extend

    if len(missing) > 0:
        missing_df = pd.concat(missing)
        print("[grpChk] The following cases are missing group assignments:")
        print(missing_df[["MICS_ID", "PNI_ID", "SES", "grp", "grp_detailed", 
            "Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)", 
            "Lateralization of epileptogenic focus"]])
        return False
    else:
        print("[grpChk] All assigned a group")
        return True

In [ ]:
def main(sheets, save_pth=None, save_name="demo"):
    """
    Run all functions to generate demographic data for 3T-7T participants.

    input:
        sheets: list of dictionarys with source sheet information (path to sheet, key columns to extract
        save_pth: path to save the output demographic file

    outputs:
        list of IDs with paired 3T-7T data
        sheet with each row as separate session and with associated demographic information
    """

    import os
    sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code") # on mac
    #sys.path.append("/host/verges/tank/data/daniel/") # on lab computer
    from Utils import id, t1
    import tTsTGrpUtils as tsutil
    
    import importlib
    importlib.reload(id)
    importlib.reload(t1)
    importlib.relaod(tsutil)
    
    out = id.ids3T7T(sheets, save_pth=None) # determine participants with 3T and 7T data
    id_cols = out.columns

    out = id.id_visits(sheets, out, save_pth=None) # get all sessions for these participants
    
    print("[main] There are ", out['MICS_ID'].nunique(), "  unique participants and ", out.shape[0], " rows in datasheet.")
    #out.to_csv(f"{save_pth}/demo_debug_1-idVisits.csv", index=False)
   
    out = t1.demo(sheets, out, save_pth=None) # add demographic info
    out.to_csv(f"{save_pth}/demo_debug_2-demo.csv", index=False) # debug

    out = rmvNADate(out, 'Date') # remove rows with missing scan dates. If only data from one study remains after this removal, then remove participant completely.
    print(out.columns)
    if 'scanDate' in out.columns:
        out = out.drop(columns=['scanDate'])

    out = dupSES(out, uniqCols=['MICS_ID', 'PNI_ID', 'study', 'SES'], mergeCols=['Date']) # merge repeated sessions
    print("[main] After cleaning for missing scan dates, there are ", out['MICS_ID'].nunique(), " unique participants with a total of ", out.shape[0], " sessions (3T:",(out['study'] == '3T').sum(),", 7T:", (out['study'] == '7T').sum(),") in input datasheet.")
    #out.to_csv(f"{save_pth}/demo_debug_3-rmvNADate.csv", index=False) # debug
    
    # save out
    if save_pth is not None:
        import datetime

        save_name_tmp = f"{save_pth}/ids3T7T_{datetime.datetime.now().strftime('%d%b%Y')}.csv"
        toSave = out[id_cols].drop_duplicates()
        toSave.to_csv(save_name_tmp, index=False)
        print("[main] Saved list of paired ids to: ", save_name_tmp)

    dob_col = None
    for sheet in sheets: # find DOB from sheet dictionary. If multiple sheets have DOB, use the first one found
        if 'DOB' in sheet and sheet['DOB'] is not None:
            dob_col = sheet['DOB']
            #print(f"[main] Found DOB column in sheet {sheet['NAME']}: {dob_col}")
            break

    if dob_col is None:
        print("[main] WARNING. No DOB column found in any input sheet. Age will not be computed.")
    
    out = tsutil.stdizeNA(out)
    out = mergeCols(out)
    print("[main] Merged columns with the same name. Current columns (sorted): \n\t", sorted(out.columns.tolist(), key=lambda x: x.lower()))

    # Fill missing demo variables for each participant, warn if multiple unique values exist
    demo_vars = ['dob', 'sex', 'gender', 'handedness', 'ethnicity'] # these demo vars should be key values in the sheet dictionaries
    print("[main] Filling missing demographic values for variables: ", demo_vars)

    for var in demo_vars:
        out = carryVals(out, var)

    out = t1.dateDif(out, [dob_col, "Date"], "age", save=False) # compute age
    
    out = group(out, out_col="grp", ID_col="MICS_ID", save_pth=None) # assign high level groups
    out = group(out, out_col="grp_detailed", ID_col="MICS_ID", save_pth=None) # assign detailed groups
    # check that all participants assigned a group
    grpChk(out, grp_cols=['grp', 'grp_detailed'])

    if save_pth is not None:
        import datetime

        save_name = f"{save_pth}/{save_name}_{datetime.datetime.now().strftime('%d%b%Y')}.csv"
        out.to_csv(save_name, index=False)
        print("[main] Saved to: ", save_name)
    else:
        print("[main] WARNING. Not saving demographics sheet to file. To save, please provide a path to save_pth")
        
    return out

## II. Make participant demographics sheet

In [270]:
src_dir = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/PT/sources" # path to directory with source pt sheets

# For each sheet, must define NAME, PATH, SHEET, ID_7T, ID_3T. 
# All other keys are those to be extracted.
# The same variables should have the same key names across sheets.

PNI = {
    'NAME': 'PNI',
    'PATH': f'{src_dir}/MICA_PNI_27Aug2025.xlsx', # 7T controls
    'SHEET': 'all', # name of sheet in file
    'ID_7T': 'ID_PNI', 
    'ID_3T': 'ID_MICs',
    'Ses_7T': 'session',
    'Date_7T': 'scanDate',
    'study': '7T',
    'DOB': 'dob',
    'Sex': 'sex',
    'Gender': 'gender',
    'Hand': 'handedness',
    'Eth': 'ethnicity',
    'Language': 'language',
    'Job': 'employment',
    'Edu': 'education',
    'LastSz': 'lastSeizure'
}

MICs = {
    'NAME': 'MICs',
    'PATH': f'{src_dir}/MICA-MTL-3T_27Aug2025.xlsx', # 3T controls
    'SHEET': 'Sheet1', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'Study_name',
    'Ses_3T': 'Visit',
    'Date_3T': 'Scan_Date (D.M.Y)',
    'study': '3T',
    'Hand': 'Handed', 
    'Sex': 'AssignedSex',
    'Gender': 'GenderIdentity',
    'Height': 'HeightApprox',
    'Weight': 'WeightApprox',
    'Eth': 'Ethnicity',
    'Job': 'Employ',
    'Edu': 'YoE',
    'LastSz': 'Last seizure'
}

Clin = {
    'NAME': 'Clin',
    'PATH': f'{src_dir}/Clinical_27Aug2025.xlsx',
    'SHEET': 'clinical-database-detailed', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'participant_id',
    'Date_3T': None,
    'Gender': 'Gender',
    'Hand': 'Handedness',
    'Language': 'Language',
    'Job': 'Employment',
    'Edu': 'Education',
    'EpilepsyDxILAE': 'Epilepsy diagnosis based on ILAE',
    'EpilepsyClass': 'Epilepsy classification:Focal,Generalized',
    'FocusLat': 'Lateralization of epileptogenic focus',
    'FocusConfirmed': 'Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)',
    'EMUDischargeDx': 'Dx at EMU discharge ',
    'EMUAdmissionDate': 'EMU admission date(dd-mm-yy)',
    'AdmissionDuration': 'Duration of admission',
    'EpilepsyRiskFactors': 'Risk factors for epilepsy',
    'SeizureOnsetYr': 'Seizure onset (yr)',
    'DrugResistant': 'Drug resistant epilepsy at time of EMU admission',
    'NumASMsPrior': '# of ASMs prior current EMU admission',
    'PrevASMs': 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission',
    'NumASMOnAdmission': '# of ASM on admission',
    'ASMsOnAdmission': 'ASMs  on admission (name, doses (mg per day)',
    'GeneticTest': 'Genetic test (year,results)',
    'FDGPET': 'FDG.PET',
    'BaselineMRI': 'Baseline MRI (year,results)',
    'InvasiveExplorations': 'Invasive explorations (Y/N)',
    'NumSurgicalResections': '# of surgical resection/thermocoagulatin',
    'SurgicalResectionDateSite': 'Surgical resection date and site',
    'Histopathology': 'Histopatholgy',
    'Engel6mo': 'Engel classification (seizure outcomes at the 6 month )',
    'Engel1yr': 'Engel classification (seizure outcomes after 1 year from surgical resection)',
    'ILAEOutcome1yr': 'ILAE outcome after surgical resection by 1 yr',
    'NeuromodDevices': 'Neuromodulation devices'
    }


sheets = [PNI, MICs, Clin]
correspSheets = [PNI]

In [ ]:
save_pth = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/outputs"
demo = main(sheets, save_pth=save_pth)


[ID_3T7T] Extracting IDs for participants with both 3T and 7T IDs
	Loading: PNI
	Skipping: MICs (missing ID_7T or ID_3T key)
	Skipping: Clin (missing ID_7T or ID_3T key)
[id_visits] Extracting visits for IDs in list
	Skipping: Clin (missing Ses_7T and Ses_3T key)


	PNI
		Extracting cols: ['ID_MICs', 'ID_PNI', 'scanDate', 'session']
	MICs
		Extracting cols: ['Study_name', 'Scan_Date (D.M.Y)', 'Visit']
		Finding corresponding ID_7T
[main] There are  63   unique participants and  203  rows in datasheet.
[demo] Retrieving demographics data.
	Overlapping: ['Sex', 'Gender', 'Hand', 'Eth', 'Language', 'Job', 'Edu', 'LastSz', 'Date_3T']
	PNI
		merge_keys: ['ID_7T', 'ID_3T', 'Ses_7T']
	MICs
		merge_keys: ['ID_3T', 'Ses_3T']
	Clin
		merge_keys: ['ID_3T']
	Removing empty columns: ['YoE', 'WeightApprox', 'AssignedSex', 'Handed', 'Employ', 'Scan_Date (D.M.Y)', 'GenderIdentity', 'HeightApprox', 'Ethnicity'] 
[rmvNADate] No rows with missing scan dates found.
Index(['MICS_ID', 'PNI_ID', 'study', 'SE

/var/folders/jz/knj2s5ld08xb9qtkljpdcl9r0000gn/T/ipykernel_78804/156988967.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[var] = df.groupby('MICS_ID')[var].transform(lambda x: x.ffill().bfill())
/var/folders/jz/knj2s5ld08xb9qtkljpdcl9r0000gn/T/ipykernel_78804/156988967.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[var] = df.groupby('MICS_ID')[var].transform(lambda x: x.ffill().bfill())
/var/folders/jz/knj2s5ld08xb9qtkljpdcl9r0000gn/T/ipykernel_78804/156988967.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, 

In [277]:
# Duplicate row check

print("-"*40)
# Find duplicated rows based on MICS_ID, PNI_ID, study, SES
dupes = demo.duplicated(subset=['MICS_ID', 'PNI_ID', 'study', 'SES'], keep=False)
if dupes.any():
    print("Duplicated rows based on ['MICS_ID', 'PNI_ID', 'study', 'SES']:")
    display(demo.loc[dupes, ['MICS_ID', 'PNI_ID', 'study', 'SES'] + [col for col in demo.columns if col not in ['MICS_ID', 'PNI_ID', 'study', 'SES']]])
else:
    print("No duplicated rows found for ['MICS_ID', 'PNI_ID', 'study', 'SES'].")


----------------------------------------
No duplicated rows found for ['MICS_ID', 'PNI_ID', 'study', 'SES'].


In [293]:
# Calculate max and median number of sessions per participant for each group and study
def max_sessions(df, group, study):
    subset = df[(df[grp_ofInterest] == group) & (df['study'] == study)]
    if subset.empty:
        return 0
    return subset.groupby('MICS_ID').size().max()

def median_sessions(df, group, study):
    subset = df[(df[grp_ofInterest] == group) & (df['study'] == study)]
    if subset.empty:
        return 0
    return subset.groupby('MICS_ID').size().median()

# Num of participants by group
grp_ofInterest = 'grp_detailed'
print("-"*40)
# Show number of unique participants per detailed group
# Calculate unique participants, number of 3T sessions, and number of 7T sessions per group
group_summary = (
    demo.groupby(grp_ofInterest)
    .agg(
        num_px=('MICS_ID', 'nunique'),
        num_ses_3T=('study', lambda x: ((x == '3T')).sum()),
        num_ses_7T=('study', lambda x: ((x == '7T')).sum()),
    )
)

group_summary['max_ses_3T'] = group_summary.index.map(lambda g: max_sessions(demo, g, '3T'))
group_summary['max_ses_7T'] = group_summary.index.map(lambda g: max_sessions(demo, g, '7T'))
group_summary['median_ses_3T'] = group_summary.index.map(lambda g: median_sessions(demo, g, '3T'))
group_summary['median_ses_7T'] = group_summary.index.map(lambda g: median_sessions(demo, g, '7T'))
# Add a total row at the end with sums for participant/session counts, leave median/max empty
total_row = {
    'num_px': group_summary['num_px'].sum(),
    'num_ses_3T': group_summary['num_ses_3T'].sum(),
    'num_ses_7T': group_summary['num_ses_7T'].sum(),
    'max_ses_3T': '',
    'max_ses_7T': '',
    'median_ses_3T': '',
    'median_ses_7T': ''
}

group_summary = pd.concat([group_summary, pd.DataFrame([total_row], index=['TOTAL'])])

group_summary = group_summary.sort_values('num_px', ascending=False)
print(group_summary)
# save to csv

output_csv = os.path.join(save_pth, f"pxSummary_3T7T_{pd.Timestamp.now().strftime('%d%b%Y')}.csv")
print(f"Saved participant summary to {output_csv}")
group_summary.to_csv(output_csv, header=True)

----------------------------------------
                                                                                         num_px  \
TOTAL                                                                                        63   
CTRL                                                                                         20   
TLE_R                                                                                         9   
TLE_L                                                                                         7   
FLE_L                                                                                         4   
UKN_U                                                                                         4   
FLE_R                                                                                         3   
PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=Focal                                         3   
TLE_BL                                                              


# Get summary statistics by category

In [260]:
# print number per group, median age, number of 3T and 7T sessions in each group
#demo['grp'].value_counts()
# count unique participants per group
demo.groupby('grp')['MICS_ID'].nunique().sort_values(ascending=False)

grp
TLE                                                            21
CTRL                                                           20
FLE                                                             7
UKN                                                             7
PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=Focal           3
MFCL                                                            1
PATTERN NOT RECOGNIZED: lobe=IGE, lat=IGE, MFCL=Generalized     1
PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=nan             1
PLE                                                             1
PQLE                                                            1
Name: MICS_ID, dtype: int64

In [45]:
# print all grp=TLE with laterlaization
demo[demo['grp'] == 'TLE'][['MICS_ID', 'PNI_ID', 'SES', 'grp', 'grp_detailed', 'Lateralization of epileptogenic focus']].drop_duplicates().sort_values(['grp', 'grp_detailed', 'MICS_ID'])

,MICS_ID,PNI_ID,SES,grp,grp_detailed,Lateralization of epileptogenic focus
63,PX153,PNE009,a1,TLE,TLE_BL,BL
64,PX153,PNE009,a2,TLE,TLE_BL,BL
141,PX153,PNE009,01,TLE,TLE_BL,BL
57,PX176,PNE004,a1,TLE,TLE_BL,BL
151,PX176,PNE004,01,TLE,TLE_BL,BL
72,PX200,PNE015,a1,TLE,TLE_BL,BL
94,PX200,PNE015,a2,TLE,TLE_BL,BL
160,PX200,PNE015,01,TLE,TLE_BL,BL
65,PX168,PNE010,a1,TLE,TLE_L,L
146,PX168,PNE010,01,TLE,TLE_L,L


In [21]:
demo[['grp', 'study', 'SES']].drop_duplicates().sort_values(['grp', 'study', 'SES'])

,grp,study,SES
178,,3T,01
90,,7T,a1
101,CTRL,3T,01
104,CTRL,3T,02
115,CTRL,3T,03
170,CTRL,3T,04
1,CTRL,7T,01
3,CTRL,7T,02
2,CTRL,7T,03
4,CTRL,7T,04


In [12]:
# get median age per study in each group
demo.groupby(['grp', 'study'])['age'].median().sort_index(level='grp')

grp   study
CTRL  3T       28.284736
      7T       28.861054
FLE   3T       33.519507
      7T       28.969199
MFCL  3T       34.135524
      7T       34.214921
TLE   3T       30.970568
      7T       30.781656
UKN   3T       31.735797
      7T       31.786448
Name: age, dtype: float64

In [13]:
# show histograms for each group

def plot_histogram(data, group_col, age_col, title):
    """
    Overlay histograms of age distribution for each group in the DataFrame.

    Parameters:
    data (pd.DataFrame): DataFrame containing the data.
    group_col (str): Column name for the group classification.
    age_col (str): Column name for the age values.
    title (str): Title for the plot.
    """
    import matplotlib.pyplot as plt
    import pandas as pd

    plt.figure(figsize=(10, 6))

    # Convert age column to numeric, coerce errors to NaN
    ages = pd.to_numeric(data[age_col], errors='coerce')

    # Overlay histograms for each group
    groups = data[group_col].unique()
    for grp in groups:
        subset = data[data[group_col] == grp]
        age_vals = pd.to_numeric(subset[age_col], errors='coerce').dropna()
        plt.hist(age_vals, bins=10, alpha=0.5, label=grp)

    plt.xlabel('Age')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_histogram(demo, 'grp_highLvl', 'age', 'Age Distribution by Group')


KeyError: 'grp_highLvl'

<Figure size 1000x600 with 0 Axes>

# Create table 1

In [ ]:
import pandas as pd

df_pth = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/demo_23May2025.csv"
df = pd.read_csv(df_pth, dtype=str)
df.columns

In [ ]:
columns_to_keep = ['MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'handedness', 'education', 'gender', 'ethnicity', 'employment', 'sex', 'age', 'grp', 'Lateralization of epileptogenic focus', 'Dx at EMU discharge ', 'grp_detailed']
df_short = df[columns_to_keep]
print(df_short.shape)
df_short['age'] = pd.to_numeric(df_short['age'], errors='coerce')
df_short.head()

In [ ]:
# print rows for HC154
df_short[df_short['MICS_ID'] == 'HC154']
# remove rows with MICS_ID == 'HC154'
df_short = df_short[df_short['MICS_ID'] != 'HC154']

In [ ]:
import numpy as np
import pandas as pd

# for repeated rows, values in columns other than Date, age and study should be the same. If NaN then replace with the first non-NaN value in the group
df_short = (
    df_short.groupby('MICS_ID', group_keys=False)
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
)
df_short = df_short.infer_objects(copy=False)

# keep most recent session only (via Date column)
df_3T = df_short[df_short['study'] == '3T']
df_3T = df_3T.sort_values(by='Date').drop_duplicates(subset='MICS_ID', keep='last')
df_3T = df_3T.reset_index(drop=True)
df_3T = df_3T.sort_values(by='MICS_ID')


df_7T = df_short[df_short['study'] == '7T']
df_7T = df_7T.sort_values(by='Date').drop_duplicates(subset='MICS_ID', keep='last')
df_7T = df_7T.reset_index(drop=True)
df_7T = df_7T.sort_values(by='MICS_ID')

print(f"3T shape: {df_3T.shape}")
print(f"7T shape: {df_7T.shape}")
print(f"Unique MICS_IDs in 3T: {df_3T['MICS_ID'].nunique()} (IDs: {df_3T['MICS_ID'].unique()})")
print(f"Unique MICS_IDs in 7T: {df_7T['MICS_ID'].nunique()} (IDs: {df_7T['MICS_ID'].unique()})")

In [ ]:
df_3T

In [ ]:
df_7T

In [ ]:
df_clean = pd.concat([df_3T, df_7T], ignore_index=True)
df_clean


In [ ]:
# compute date difference between 3T and 7T scans by individual
df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
df_clean['Time between scans (mths)'] = np.nan
# for each unique MICS_ID, compute the difference between the 3T and 7T dates. Return this value to both rows
for mics_id in df_clean['MICS_ID'].unique():
    dates = df_clean[df_clean['MICS_ID'] == mics_id]['Date']
    
    date_diff_months = abs((dates.max() - dates.min()).days / 30.44)  # average days per month, absolute value
    df_clean.loc[df_clean['MICS_ID'] == mics_id, 'Time between scans (mths)'] = date_diff_months

df_clean.sort_values(by='MICS_ID')

In [ ]:
# make histogram of time between scans
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['Time between scans (mths)'].dropna(), bins=30, kde=True)
plt.title('Time Between 3T and 7T Scans')

mean_val = df_clean['Time between scans (mths)'].mean()
median_val = df_clean['Time between scans (mths)'].median()

plt.axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
plt.axvline(median_val, color='green', linestyle='-.', label=f'Median: {median_val:.2f}')

plt.xlabel('Time (months)')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.show()

In [ ]:
def make_table(df, stat='mdn', grp='grp', age='age', sex='sex', scan_dif='Time between scans (mths)', edu='education', eth='ethnicity', hand='handedness'):
    """
    Create a summary statistics table for the given DataFrame.
    """
    table = pd.DataFrame(columns=df[grp].unique())

    grp_values = list(df[grp].unique()) + ['ALL PX']
    for grp_value in grp_values:
        if grp_value == 'ALL PX':
            sub_df = df[df[grp] != 'CTRL'].copy()
        else:
            sub_df = df[df[grp] == grp_value].copy()

        # counts
        n_individuals = sub_df['MICS_ID'].nunique()
        table.loc['n', grp_value] = n_individuals

        # Ensure numeric for stats
        sub_df[age] = pd.to_numeric(sub_df[age], errors='coerce')
        sub_df[scan_dif] = pd.to_numeric(sub_df[scan_dif], errors='coerce')

        # Numerical: age (per study)
        for study in sub_df['study'].unique():
            study_mask = sub_df['study'] == study
            if stat == 'mdn':
                age_mdn = sub_df.loc[study_mask, age].median()
                age_iqr = sub_df.loc[study_mask, age].quantile(0.75) - sub_df.loc[study_mask, age].quantile(0.25)
                row_label = f'age_{study} (mdn (IQR))'
                table.loc[row_label, grp_value] = f"{age_mdn:.1f} ({age_iqr:.1f})"
            elif stat == 'mean':
                age_mean = sub_df.loc[study_mask, age].mean()
                age_std = sub_df.loc[study_mask, age].std()
                row_label = f'age_{study} (mean (std))'
                table.loc[row_label, grp_value] = f"{age_mean:.1f} ({age_std:.0f})"

        # remove repeated rows based on MICS_ID
        sub_df = sub_df.drop_duplicates(subset='MICS_ID')

        # Numerical: Time between scans (mths)
        if stat == 'mdn':
            tbs_mdn = sub_df[scan_dif].median()
            tbs_iqr = sub_df[scan_dif].quantile(0.75) - sub_df[scan_dif].quantile(0.25)
            row_label = 'Time between scans (mths) (mdn (IQR))'
            table.loc[row_label, grp_value] = f"{tbs_mdn:.1f} ({tbs_iqr:.1f})"
        elif stat == 'mean':
            tbs_mean = sub_df[scan_dif].mean()
            tbs_std = sub_df[scan_dif].std()
            row_label = 'Time between scans (mths) (mean (std))'
            table.loc[row_label, grp_value] = f"{tbs_mean:.1f} ({tbs_std:.1f})"

        # Cat: sex
        sub_df.loc[:, sex] = sub_df[sex].str.lower().map({'f': 'female', 'm': 'male'})
        n_females = (sub_df[sex] == 'female').sum()
        n_males = (sub_df[sex] == 'male').sum()
        n_total = sub_df[sex].notna().sum()
        pct_females = (n_females / n_total * 100) if n_total > 0 else 0
        row_label = 'sex (F:M (% F))'
        row_label_count = 'sex (F:M (%F))'
        table.loc[row_label_count, grp_value] = f"{n_females}:{n_males} ({pct_females:.0f}%)"

        # Cat: education
        edu_counts = sub_df[edu].value_counts(dropna=False)
        edu_str = ', '.join([f"{k}:{v}" for k, v in edu_counts.items()])
        row_label = 'education (count)'
        table.loc[row_label, grp_value] = edu_str

        # Cat: Handedness
        hand_counts = sub_df[hand].value_counts(dropna=False)
        n_left = hand_counts.get('L', 0)
        n_right = hand_counts.get('R', 0)
        hand_str = f"{n_left}:{n_right}"
        row_label = 'handedness (L:R)'
        table.loc[row_label, grp_value] = hand_str

        # Cat: lateralization of epileptogenic focus
        if grp_value != 'CTRL':
            lat_counts = sub_df['Lateralization of epileptogenic focus'].value_counts(dropna=False)
            n_tot = lat_counts.sum()
            # Report as L:R:unknown
            n_left = lat_counts.get('L', 0)
            n_left += lat_counts.get('L ', 0)  # Handle trailing space
            n_right = lat_counts.get('R', 0)
            n_right += lat_counts.get('R ', 0)  # Handle trailing space
            n_right += lat_counts.get('right', 0)  # Handle 'right' case
            n_left += lat_counts.get('left', 0)  # Handle 'left' case
            # Count L>R or R>L and add to L or R accordingly
            n_unclear = lat_counts.get('unclear', 0)
            n_unclear += lat_counts.get('unclear ', 0)
            
            print(f"Tot: {n_tot}, sum: {n_left + n_right + n_unclear}")
            
            if n_left + n_right + n_unclear != n_tot:
                print(f"\t {n_left + n_right + n_unclear} != {n_tot} for {grp_value}")
                n_bl = lat_counts.get('BL', 0)

                if n_bl > 0:
                    print("BL found in Lateralization of epileptogenic focus")
                    print(f"R, L counts: {n_right}, {n_left}")
                    # Subset rows with 'BL' in 'Lateralization of epileptogenic focus'
                    bl_rows = sub_df[sub_df['Lateralization of epileptogenic focus'] == 'BL']
                    # Search 'Dx at EMU discharge ' for 'L>' or 'R>' and add to counts
                    n_bl_l = bl_rows['Dx at EMU discharge '].astype(str).str.contains('L>').sum()
                    n_bl_r = bl_rows['Dx at EMU discharge '].astype(str).str.contains('R>').sum()
                    n_left += n_bl_l
                    n_right += n_bl_r
                    # Count all other BL cases that are not clearly L> or R>
                    n_bl_u = n_bl - (n_bl_l + n_bl_r)
                    print(f"R, L counts: {n_right}, {n_left}")
            else:
                n_bl_u = 0

            lat_str = f"{n_left}:{n_right}:{n_bl_u}:{n_unclear}"
        
        else:
            lat_str = ''
        
        row_label = 'Lateralization (L:R:BL (unclear dominance):unclear)'
        table.loc[row_label, grp_value] = lat_str

    return table


In [ ]:
t1 = make_table(df_clean, stat='mean', grp='grp')

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
t1
